In [207]:
import numpy as np
import pandas as pd
import datetime

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import dill as pickle

import warnings
warnings.simplefilter('ignore')

In [118]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    """
    The class provides basic functionality for retrieving
    a subset of columns from the dataset.
    """
    
    features = ['ABS (антиблокировочная система)',
         'AUX/iPod',
         'Bluetooth',
         'CD/MP3 проигрыватель',
         'ESP (система поддержания динамической стабильности)',
         'USB',
         'Автозапуск двигателя',
         'Антипробуксовочная система',
         'Датчик дождя',
         'Иммобилайзер',
         'Камера заднего вида',
         'Климат-контроль',
         'Кондиционер',
         'Контроль мертвых зон на зеркалах',
         'Круиз-контроль',
         'Ксеноновые фары',
         'Легкосплавные диски',
         'Люк',
         'Материал салона',
         'Мультимедийный экран',
         'Обогрев зеркал',
         'Обогрев лобового стекла',
         'Обогрев руля',
         'Обогрев сидений',
         'Панорамная крыша',
         'Парктроники',
         'Подушки безопасности боковые',
         'Подушки безопасности задние',
         'Подушки безопасности передние',
         'Противотуманные фары',
         'Рейлинги на крыше',
         'Светодиодные фары',
         'Сигнализация',
         'Управление мультимедиа с руля',
         'Фаркоп',
         'Цвет салона - темный',
         'Штатная навигация',
         'Электрорегулировка сидений',
         'Электростеклоподъемники задние',
         'Электростеклоподъемники передние']
    
    def __init__(self, feature_names=features):
        """
        Initialize class instance by setting
        a list of columns to retrieve from the dataset.
        """
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
        self.feature_names = feature_names
        
    def fit(self, X, y=None):
        """
        Fit FeatureSelector to X, but really do nothing.
        Return self.
        """
        return self
    
    def transform(self, X, y=None):
        """
        Transform X using feature selection. 
        Return column-subset of X.
        """
        return X[self.feature_names]

In [119]:
class ToIntTransformer(BaseEstimator, TransformerMixin):
    """
    Required columns: volume, run, name, year.
    For update data.
    """
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        self.to_int(X, 'Объем')
        self.to_int(X, 'Пробег')
        self.year_to_old(X)
        
        return X.drop(['Название', 'Год'], axis=1).values
        
    def year_to_old(self, X):
        X['Возраст'] = (datetime.datetime.now().year - X['Год']).astype('int8')
        
    def to_int(self, X, column_name):
        X[column_name] = X[column_name].astype('int32')
        

In [120]:
class FeatureGenerator(BaseEstimator, TransformerMixin):
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X
            

In [121]:
class CatTransformer(BaseEstimator, TransformerMixin):
    """
    Required columns: cuzov, fuel, name, region.
    """    
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
#         self.cut(X, ['cuzov', 'fuel'])
        self.fix_names(X)
        return X
    
    def cut(self, X, column_names):
        for col in column_names:
            X[col] = X[col].apply(lambda x: x.split()[0])
        
    def fix_names(self, df):
        value_count = df['Название'].value_counts()
        df['Название'] = df['Название'].apply(lambda x: x if value_count[x] > 100 else 'другая')
        

In [178]:
from_file = 'data/full_vis.csv'

In [179]:
avto = pd.read_csv(from_file, sep=';')

In [180]:
avto
# datetime.datetime.now().year

,Название,Цена,Регион,Год,Состояние,Пробег,Тип топлива,Объем,Цвет,Тип кузова,...,Рейлинги на крыше,Светодиодные фары,Сигнализация,Управление мультимедиа с руля,Фаркоп,Цвет салона - темный,Штатная навигация,Электрорегулировка сидений,Электростеклоподъемники задние,Электростеклоподъемники передние
0,BMW X1,48019,Минск,2016,с пробегом,181000,дизель,2000,другой,внедорожник 5 дв,...,0,0,1,1,1,0,1,1,1,1
1,Citroen Xantia,4950,Гомельская обл.,1998,с пробегом,363400,дизель,2100,другой,хэтчбек 5 дв,...,0,0,0,1,0,1,0,0,0,1
2,Mercedes-Benz E-Класс,20297,Гродненская обл.,2003,с пробегом,307000,дизель,2200,черный,седан,...,0,0,0,1,0,1,0,1,1,1
3,Peugeot 605,1485,Минск,1996,с пробегом,107000,дизель,2100,зеленый,седан,...,0,0,1,0,0,1,0,0,0,1
4,Volkswagen Passat,9777,Гомельская обл.,2000,с пробегом,252475,дизель,1900,черный,универсал,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68313,Audi A6,12376,Гродненская обл.,2001,с пробегом,345000,дизель,2500,зеленый,универсал,...,1,0,1,0,1,0,0,0,0,0
68314,Fiat 500,20297,Брестская обл.,2012,с пробегом,79000,бензин,900,белый,хэтчбек 3 дв,...,0,0,0,1,0,0,0,0,0,1
68315,Nissan Primera,10520,Минск,2004,с пробегом,273538,бензин,1800,серый,седан,...,0,0,0,1,0,0,0,0,1,1
68316,BMW 3,17324,Минск,2002,с пробегом,317,бензин,2500,серый,кабриолет,...,0,0,0,1,0,1,0,1,1,1


In [208]:
X = avto.drop(['Цена'], axis=1)
y = avto['Цена'].apply(lambda x: int(str(x).replace(' ', '')))
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2, random_state=42)

feat_pipeline = Pipeline(
    steps=[
        ('feat_selector', FeatureSelector(FeatureSelector().feature_names)),
        ('feat_generator', FeatureGenerator()),
        ('feat_encoder', OneHotEncoder())
    ]
)

int_pipeline = Pipeline(
    steps=[
        ('int_selector', FeatureSelector(['Объем', 'Название', 'Пробег', 'Год'])),
        ('int_transformer', ToIntTransformer())
    ]
)

cat_pipeline = Pipeline(
    steps=[
        ('cat_selector', FeatureSelector(['Тип кузова', 'Тип топлива', 'Название', 'Регион', 'Цвет'])),
        ('cat_transformer', CatTransformer()),
        ('cat_encoder', OneHotEncoder())
    ]
)

no_proc_pipeline = Pipeline(
    steps=[
        ('no_proc_selector', FeatureSelector(['Привод', 'Состояние', 'Трансмиссия'])),
        ('no_proc_imputer', SimpleImputer(strategy='most_frequent')),
        ('no_proc_encoder', OneHotEncoder())
    ]
)

pipeline = FeatureUnion(transformer_list= [
    ('feat', feat_pipeline),
    ('int', int_pipeline),
    ('cat', cat_pipeline),
    ('no_proc', no_proc_pipeline)
])

In [209]:
X_train
pipeline

FeatureUnion(n_jobs=None,
             transformer_list=[('feat',
                                Pipeline(memory=None,
                                         steps=[('feat_selector',
                                                 FeatureSelector(feature_names=['ABS '
                                                                                '(антиблокировочная '
                                                                                'система)',
                                                                                'AUX/iPod',
                                                                                'Bluetooth',
                                                                                'CD/MP3 '
                                                                                'проигрыватель',
                                                                                'ESP '
                                                                          

### Random forest

In [210]:
pip_rand_f = Pipeline(
    steps=[
        ('preprocessing', pipeline),
        ('model', RandomForestRegressor(
                        max_depth=20, 
                        min_samples_split=5, 
                        min_samples_leaf=1,
                        max_features=None,
                        random_state=42,
                        n_estimators=150,
                        n_jobs=-1
        ))
    ]
)

In [ ]:
pip_rand_f.fit(X_train, y_train)

In [ ]:
y_pred = pip_rand_f.predict(X_test)
print(metrics.r2_score(y_test, y_pred))

In [ ]:
pip_rand_f['model'].save_model('model_rand_forest_score_095846.cbm')

In [130]:
pip_rand_f['model'].feature_importances_

array([2.28512512e-04, 1.91454526e-04, 2.75914505e-04, 2.90079027e-04,
       2.08329866e-04, 2.09543752e-04, 4.34887335e-04, 4.07737722e-04,
       5.56669651e-04, 4.78414079e-04, 1.69514554e-04, 1.82202756e-04,
       6.04655684e-04, 5.56872417e-04, 1.96941376e-04, 2.15379525e-04,
       1.18352291e-03, 1.25529994e-03, 2.97466400e-04, 3.42060676e-04,
       5.19839343e-04, 5.11783775e-04, 4.68156798e-04, 5.34481523e-04,
       4.88665679e-04, 5.66324234e-04, 7.63294476e-04, 6.22991295e-04,
       4.20220080e-04, 3.87962485e-04, 1.11048071e-03, 1.17857617e-03,
       3.45395680e-04, 3.44050656e-04, 5.91338526e-04, 6.04001809e-04,
       4.08598554e-04, 3.34194017e-04, 4.67165348e-04, 9.73545604e-04,
       2.70597700e-03, 1.69024731e-03, 1.28067308e-03, 1.42688095e-03,
       2.95337178e-04, 3.03347095e-04, 8.55983628e-04, 7.43596356e-04,
       1.64052746e-03, 1.65197461e-03, 2.27886919e-04, 2.28657382e-04,
       2.66741090e-03, 2.86703452e-03, 3.83444886e-04, 3.33594202e-04,
      

In [199]:
pip_rand_f['model'].feature_importances_

AttributeError: 'Pipeline' object has no attribute 'best_params_'

In [131]:
avto

,Название,Цена,Регион,Год,Состояние,Пробег,Тип топлива,Объем,Цвет,Тип кузова,...,Рейлинги на крыше,Светодиодные фары,Сигнализация,Управление мультимедиа с руля,Фаркоп,Цвет салона - темный,Штатная навигация,Электрорегулировка сидений,Электростеклоподъемники задние,Электростеклоподъемники передние
0,BMW X1,48019,Минск,2016,с пробегом,181000,дизель,2000,другой,внедорожник 5 дв,...,0,0,1,1,1,0,1,1,1,1
1,Citroen Xantia,4950,Гомельская обл.,1998,с пробегом,363400,дизель,2100,другой,хэтчбек 5 дв,...,0,0,0,1,0,1,0,0,0,1
2,Mercedes-Benz E-Класс,20297,Гродненская обл.,2003,с пробегом,307000,дизель,2200,черный,седан,...,0,0,0,1,0,1,0,1,1,1
3,Peugeot 605,1485,Минск,1996,с пробегом,107000,дизель,2100,зеленый,седан,...,0,0,1,0,0,1,0,0,0,1
4,Volkswagen Passat,9777,Гомельская обл.,2000,с пробегом,252475,дизель,1900,черный,универсал,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68313,Audi A6,12376,Гродненская обл.,2001,с пробегом,345000,дизель,2500,зеленый,универсал,...,1,0,1,0,1,0,0,0,0,0
68314,Fiat 500,20297,Брестская обл.,2012,с пробегом,79000,бензин,900,белый,хэтчбек 3 дв,...,0,0,0,1,0,0,0,0,0,1
68315,Nissan Primera,10520,Минск,2004,с пробегом,273538,бензин,1800,серый,седан,...,0,0,0,1,0,0,0,0,1,1
68316,BMW 3,17324,Минск,2002,с пробегом,317,бензин,2500,серый,кабриолет,...,0,0,0,1,0,1,0,1,1,1


#### GS_RForest

In [132]:
param_grid = [
    {
        'max_depth': [ 8, 10, 15],
        'n_estimators': [100, 150],
        'min_samples_split': [3, 5]
    }
]

pip_rf = Pipeline(
    steps=[
        ('preprocessing', pipeline),
        ('rf', GridSearchCV(RandomForestRegressor(
                        random_state=42,
                        n_jobs=-1
        ), param_grid, scoring='r2', cv=5))
    ]
)

In [133]:
pip_rf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessing',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('feat',
                                                 Pipeline(memory=None,
                                                          steps=[('feat_selector',
                                                                  FeatureSelector(feature_names=['ABS '
                                                                                                 '(антиблокировочная '
                                                                                                 'система)',
                                                                                                 'AUX/iPod',
                                                                                                 'Bluetooth',
                                                                                                 'CD/MP3 '
                                

In [134]:
y_true, y_pred = y_test, pip_rf.predict(X_test)
print(metrics.r2_score(y_true, y_pred))

0.9519112990269769


In [206]:
filename = 'model_GS_RF_score_095196.pk'

with open('models/'+filename, 'wb') as file:
    pickle.dump(pip_rf['rf'], file)
    
#     pip_rf['rf'].save_model('model_GS_RF_score_095196.cbm')

In [135]:
pip_rf[1].best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=3,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

#### GS_Tree

In [136]:
param_grid = {"criterion": ["mse", "mae"],
              "min_samples_split": [5, 10, 20],
              "max_depth": [2, 5, 8, 10],
               "min_samples_leaf": [20, 40, 100],
               "max_leaf_nodes": [None, 5, 20, 10],
              }
pip_tree = Pipeline(
    steps=[
        ('preprocessing', pipeline),
        ('tree', GridSearchCV(DecisionTreeRegressor(
                        random_state=42
        ), param_grid, scoring='r2', cv=5))
    ]
)

In [ ]:
pip_tree.fit(X_train, y_train)
y_true, y_pred = y_test, pip_rf.predict(X_test)
print(metrics.r2_score(y_true, y_pred))

In [ ]:
pip_tree[1].best_estimator_

### cat_boost

In [143]:
import catboost
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [171]:
t = [elem for elem in dct]
t

['ткань',
 'натуральная кожа',
 'велюр',
 'комбинированный',
 'искусственная кожа',
 'алькантара']

In [183]:
X = avto.drop(['Цена'], axis=1)
y = avto['Цена'].apply(lambda x: int(str(x).replace(' ', '')))
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2)

In [184]:
avto

,Название,Цена,Регион,Год,Состояние,Пробег,Тип топлива,Объем,Цвет,Тип кузова,...,Рейлинги на крыше,Светодиодные фары,Сигнализация,Управление мультимедиа с руля,Фаркоп,Цвет салона - темный,Штатная навигация,Электрорегулировка сидений,Электростеклоподъемники задние,Электростеклоподъемники передние
0,BMW X1,48019,Минск,2016,с пробегом,181000,дизель,2000,другой,внедорожник 5 дв,...,0,0,1,1,1,0,1,1,1,1
1,Citroen Xantia,4950,Гомельская обл.,1998,с пробегом,363400,дизель,2100,другой,хэтчбек 5 дв,...,0,0,0,1,0,1,0,0,0,1
2,Mercedes-Benz E-Класс,20297,Гродненская обл.,2003,с пробегом,307000,дизель,2200,черный,седан,...,0,0,0,1,0,1,0,1,1,1
3,Peugeot 605,1485,Минск,1996,с пробегом,107000,дизель,2100,зеленый,седан,...,0,0,1,0,0,1,0,0,0,1
4,Volkswagen Passat,9777,Гомельская обл.,2000,с пробегом,252475,дизель,1900,черный,универсал,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68313,Audi A6,12376,Гродненская обл.,2001,с пробегом,345000,дизель,2500,зеленый,универсал,...,1,0,1,0,1,0,0,0,0,0
68314,Fiat 500,20297,Брестская обл.,2012,с пробегом,79000,бензин,900,белый,хэтчбек 3 дв,...,0,0,0,1,0,0,0,0,0,1
68315,Nissan Primera,10520,Минск,2004,с пробегом,273538,бензин,1800,серый,седан,...,0,0,0,1,0,0,0,0,1,1
68316,BMW 3,17324,Минск,2002,с пробегом,317,бензин,2500,серый,кабриолет,...,0,0,0,1,0,1,0,1,1,1


Train


In [185]:
ctbst = catboost.CatBoostRegressor(cat_features=['Тип кузова', 'Тип топлива', 'Название', 'Регион',
                                                 'Цвет', 'Привод', 'Состояние', 'Трансмиссия', 'Материал салона'],
                                   od_pval=1e-5)

X = avto.drop(['Цена'], axis=1)
y = avto['Цена'].apply(lambda x: int(str(x).replace(' ', '')))
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2, random_state=42)


In [186]:
ctbst.fit(X_train, y_train)

0:	learn: 21979.0798737	total: 279ms	remaining: 4m 38s
1:	learn: 21538.7945439	total: 374ms	remaining: 3m 6s
2:	learn: 21107.9381301	total: 467ms	remaining: 2m 35s
3:	learn: 20691.0813904	total: 552ms	remaining: 2m 17s
4:	learn: 20297.4452076	total: 642ms	remaining: 2m 7s
5:	learn: 19906.7572860	total: 752ms	remaining: 2m 4s
6:	learn: 19536.5336673	total: 875ms	remaining: 2m 4s
7:	learn: 19166.4764967	total: 971ms	remaining: 2m
8:	learn: 18824.1274917	total: 1.06s	remaining: 1m 56s
9:	learn: 18467.0044859	total: 1.17s	remaining: 1m 55s
10:	learn: 18131.8180764	total: 1.25s	remaining: 1m 52s
11:	learn: 17817.4521529	total: 1.36s	remaining: 1m 52s
12:	learn: 17512.7342196	total: 1.48s	remaining: 1m 52s
13:	learn: 17212.1277019	total: 1.58s	remaining: 1m 51s
14:	learn: 16920.2979641	total: 1.68s	remaining: 1m 50s
15:	learn: 16631.8911245	total: 1.78s	remaining: 1m 49s
16:	learn: 16375.9317461	total: 1.88s	remaining: 1m 48s
17:	learn: 16107.5084276	total: 2.02s	remaining: 1m 49s
18:	learn:

148:	learn: 7638.3020637	total: 18.9s	remaining: 1m 47s
149:	learn: 7625.8394753	total: 19s	remaining: 1m 47s
150:	learn: 7618.6723528	total: 19.1s	remaining: 1m 47s
151:	learn: 7610.6255664	total: 19.2s	remaining: 1m 47s
152:	learn: 7595.1876192	total: 19.4s	remaining: 1m 47s
153:	learn: 7585.0740957	total: 19.5s	remaining: 1m 47s
154:	learn: 7573.9517715	total: 19.7s	remaining: 1m 47s
155:	learn: 7562.3343832	total: 19.8s	remaining: 1m 47s
156:	learn: 7551.7730210	total: 20s	remaining: 1m 47s
157:	learn: 7539.1328799	total: 20.1s	remaining: 1m 47s
158:	learn: 7529.3704701	total: 20.3s	remaining: 1m 47s
159:	learn: 7517.0932376	total: 20.4s	remaining: 1m 46s
160:	learn: 7506.9142803	total: 20.5s	remaining: 1m 46s
161:	learn: 7498.5088112	total: 20.7s	remaining: 1m 46s
162:	learn: 7484.6539671	total: 20.8s	remaining: 1m 46s
163:	learn: 7471.6234424	total: 20.9s	remaining: 1m 46s
164:	learn: 7461.5743830	total: 21.1s	remaining: 1m 46s
165:	learn: 7453.2504973	total: 21.2s	remaining: 1m 

296:	learn: 6621.7453387	total: 37.4s	remaining: 1m 28s
297:	learn: 6618.7513986	total: 37.5s	remaining: 1m 28s
298:	learn: 6614.2743228	total: 37.7s	remaining: 1m 28s
299:	learn: 6608.0095887	total: 37.8s	remaining: 1m 28s
300:	learn: 6601.4350314	total: 37.9s	remaining: 1m 28s
301:	learn: 6596.8398205	total: 38s	remaining: 1m 27s
302:	learn: 6590.3394104	total: 38.2s	remaining: 1m 27s
303:	learn: 6582.7451718	total: 38.3s	remaining: 1m 27s
304:	learn: 6578.4285662	total: 38.4s	remaining: 1m 27s
305:	learn: 6574.0057332	total: 38.5s	remaining: 1m 27s
306:	learn: 6571.3039850	total: 38.7s	remaining: 1m 27s
307:	learn: 6564.0809410	total: 38.8s	remaining: 1m 27s
308:	learn: 6560.1472286	total: 38.9s	remaining: 1m 26s
309:	learn: 6558.4820595	total: 39s	remaining: 1m 26s
310:	learn: 6553.3394505	total: 39.1s	remaining: 1m 26s
311:	learn: 6549.0281182	total: 39.2s	remaining: 1m 26s
312:	learn: 6546.4738573	total: 39.3s	remaining: 1m 26s
313:	learn: 6544.2265646	total: 39.4s	remaining: 1m 

443:	learn: 6074.8399786	total: 55.9s	remaining: 1m 9s
444:	learn: 6071.9178716	total: 56s	remaining: 1m 9s
445:	learn: 6067.5577650	total: 56.1s	remaining: 1m 9s
446:	learn: 6066.1852734	total: 56.3s	remaining: 1m 9s
447:	learn: 6063.2873614	total: 56.4s	remaining: 1m 9s
448:	learn: 6061.6021930	total: 56.5s	remaining: 1m 9s
449:	learn: 6058.4024804	total: 56.6s	remaining: 1m 9s
450:	learn: 6057.1689380	total: 56.8s	remaining: 1m 9s
451:	learn: 6054.0702008	total: 56.9s	remaining: 1m 8s
452:	learn: 6052.1803941	total: 57s	remaining: 1m 8s
453:	learn: 6048.4522692	total: 57.1s	remaining: 1m 8s
454:	learn: 6047.3763752	total: 57.2s	remaining: 1m 8s
455:	learn: 6042.4003435	total: 57.3s	remaining: 1m 8s
456:	learn: 6039.6491745	total: 57.4s	remaining: 1m 8s
457:	learn: 6035.1300406	total: 57.7s	remaining: 1m 8s
458:	learn: 6031.2711961	total: 57.8s	remaining: 1m 8s
459:	learn: 6027.4684809	total: 57.9s	remaining: 1m 8s
460:	learn: 6024.4483188	total: 58.1s	remaining: 1m 7s
461:	learn: 60

593:	learn: 5696.7091595	total: 1m 15s	remaining: 51.9s
594:	learn: 5694.0905741	total: 1m 15s	remaining: 51.7s
595:	learn: 5691.2013888	total: 1m 16s	remaining: 51.6s
596:	learn: 5688.4186547	total: 1m 16s	remaining: 51.5s
597:	learn: 5687.6883235	total: 1m 16s	remaining: 51.3s
598:	learn: 5685.7271198	total: 1m 16s	remaining: 51.2s
599:	learn: 5683.2384317	total: 1m 16s	remaining: 51s
600:	learn: 5682.2515978	total: 1m 16s	remaining: 50.9s
601:	learn: 5680.7549122	total: 1m 16s	remaining: 50.8s
602:	learn: 5679.3048045	total: 1m 16s	remaining: 50.6s
603:	learn: 5677.5665806	total: 1m 17s	remaining: 50.5s
604:	learn: 5674.8482812	total: 1m 17s	remaining: 50.4s
605:	learn: 5674.3667112	total: 1m 17s	remaining: 50.2s
606:	learn: 5672.7212843	total: 1m 17s	remaining: 50.1s
607:	learn: 5672.1309926	total: 1m 17s	remaining: 50s
608:	learn: 5668.5297711	total: 1m 17s	remaining: 49.8s
609:	learn: 5665.3310365	total: 1m 17s	remaining: 49.7s
610:	learn: 5663.2428630	total: 1m 17s	remaining: 49

741:	learn: 5440.2168186	total: 1m 34s	remaining: 33s
742:	learn: 5436.1192839	total: 1m 35s	remaining: 32.9s
743:	learn: 5435.0766004	total: 1m 35s	remaining: 32.8s
744:	learn: 5432.9927993	total: 1m 35s	remaining: 32.6s
745:	learn: 5432.0917740	total: 1m 35s	remaining: 32.5s
746:	learn: 5430.7328568	total: 1m 35s	remaining: 32.4s
747:	learn: 5428.0640429	total: 1m 35s	remaining: 32.2s
748:	learn: 5426.0866608	total: 1m 35s	remaining: 32.1s
749:	learn: 5425.0046142	total: 1m 35s	remaining: 32s
750:	learn: 5424.0985471	total: 1m 36s	remaining: 31.8s
751:	learn: 5421.1970575	total: 1m 36s	remaining: 31.7s
752:	learn: 5419.1081486	total: 1m 36s	remaining: 31.6s
753:	learn: 5416.2777894	total: 1m 36s	remaining: 31.5s
754:	learn: 5414.4078373	total: 1m 36s	remaining: 31.3s
755:	learn: 5412.4860895	total: 1m 36s	remaining: 31.2s
756:	learn: 5410.4818831	total: 1m 36s	remaining: 31.1s
757:	learn: 5408.7795336	total: 1m 36s	remaining: 30.9s
758:	learn: 5406.5166053	total: 1m 37s	remaining: 30

889:	learn: 5207.7008753	total: 1m 53s	remaining: 14s
890:	learn: 5207.0876494	total: 1m 53s	remaining: 13.9s
891:	learn: 5205.5233353	total: 1m 53s	remaining: 13.7s
892:	learn: 5204.7183091	total: 1m 53s	remaining: 13.6s
893:	learn: 5202.0697560	total: 1m 53s	remaining: 13.5s
894:	learn: 5200.8583650	total: 1m 53s	remaining: 13.4s
895:	learn: 5199.3732537	total: 1m 54s	remaining: 13.2s
896:	learn: 5197.0959008	total: 1m 54s	remaining: 13.1s
897:	learn: 5195.3657805	total: 1m 54s	remaining: 13s
898:	learn: 5193.4245862	total: 1m 54s	remaining: 12.8s
899:	learn: 5192.8488670	total: 1m 54s	remaining: 12.7s
900:	learn: 5192.2100574	total: 1m 54s	remaining: 12.6s
901:	learn: 5190.4176574	total: 1m 54s	remaining: 12.5s
902:	learn: 5188.3756382	total: 1m 54s	remaining: 12.3s
903:	learn: 5187.3873472	total: 1m 54s	remaining: 12.2s
904:	learn: 5186.7208234	total: 1m 55s	remaining: 12.1s
905:	learn: 5186.3332185	total: 1m 55s	remaining: 12s
906:	learn: 5184.8058093	total: 1m 55s	remaining: 11.8

In [188]:
from sklearn.metrics import r2_score

In [189]:
r2_score(y_test, ctbst.predict(X_test))

0.9420186663926003

In [191]:
cross_val_score(ctbst, X, y, cv=5)

0:	learn: 21649.4108517	total: 105ms	remaining: 1m 44s
1:	learn: 21215.3445065	total: 202ms	remaining: 1m 40s
2:	learn: 20817.4052686	total: 314ms	remaining: 1m 44s
3:	learn: 20418.7106095	total: 417ms	remaining: 1m 43s
4:	learn: 20020.9926131	total: 536ms	remaining: 1m 46s
5:	learn: 19647.1193306	total: 670ms	remaining: 1m 51s
6:	learn: 19297.6107579	total: 808ms	remaining: 1m 54s
7:	learn: 18936.6453854	total: 926ms	remaining: 1m 54s
8:	learn: 18615.5262132	total: 1.08s	remaining: 1m 59s
9:	learn: 18270.8643791	total: 1.2s	remaining: 1m 59s
10:	learn: 17947.9928686	total: 1.33s	remaining: 1m 59s
11:	learn: 17632.7316769	total: 1.44s	remaining: 1m 58s
12:	learn: 17322.3419689	total: 1.55s	remaining: 1m 58s
13:	learn: 17026.2486050	total: 1.67s	remaining: 1m 57s
14:	learn: 16741.6897681	total: 1.79s	remaining: 1m 57s
15:	learn: 16456.6752828	total: 1.9s	remaining: 1m 57s
16:	learn: 16181.2018536	total: 2.04s	remaining: 1m 58s
17:	learn: 15916.1559480	total: 2.17s	remaining: 1m 58s
18:	

148:	learn: 7651.0438148	total: 18.1s	remaining: 1m 43s
149:	learn: 7641.3275784	total: 18.2s	remaining: 1m 43s
150:	learn: 7627.5301906	total: 18.4s	remaining: 1m 43s
151:	learn: 7619.3595438	total: 18.5s	remaining: 1m 43s
152:	learn: 7606.4110953	total: 18.6s	remaining: 1m 42s
153:	learn: 7594.8763409	total: 18.7s	remaining: 1m 42s
154:	learn: 7584.8159852	total: 18.9s	remaining: 1m 42s
155:	learn: 7571.5393577	total: 19s	remaining: 1m 42s
156:	learn: 7560.4377531	total: 19.1s	remaining: 1m 42s
157:	learn: 7552.0453933	total: 19.2s	remaining: 1m 42s
158:	learn: 7544.2776748	total: 19.4s	remaining: 1m 42s
159:	learn: 7536.0983403	total: 19.5s	remaining: 1m 42s
160:	learn: 7525.6361866	total: 19.6s	remaining: 1m 41s
161:	learn: 7518.5193016	total: 19.7s	remaining: 1m 41s
162:	learn: 7507.9551298	total: 19.8s	remaining: 1m 41s
163:	learn: 7501.2632901	total: 19.9s	remaining: 1m 41s
164:	learn: 7493.0288366	total: 20s	remaining: 1m 41s
165:	learn: 7483.3528482	total: 20.1s	remaining: 1m 

296:	learn: 6606.4426218	total: 36s	remaining: 1m 25s
297:	learn: 6605.0562367	total: 36.1s	remaining: 1m 25s
298:	learn: 6600.7466230	total: 36.3s	remaining: 1m 25s
299:	learn: 6599.1747925	total: 36.4s	remaining: 1m 24s
300:	learn: 6594.5539579	total: 36.5s	remaining: 1m 24s
301:	learn: 6587.9852692	total: 36.6s	remaining: 1m 24s
302:	learn: 6582.2253081	total: 36.8s	remaining: 1m 24s
303:	learn: 6578.7587622	total: 36.9s	remaining: 1m 24s
304:	learn: 6572.2538083	total: 37.1s	remaining: 1m 24s
305:	learn: 6568.0347890	total: 37.2s	remaining: 1m 24s
306:	learn: 6561.9754103	total: 37.3s	remaining: 1m 24s
307:	learn: 6559.0989807	total: 37.4s	remaining: 1m 24s
308:	learn: 6554.9571040	total: 37.5s	remaining: 1m 23s
309:	learn: 6551.2758720	total: 37.6s	remaining: 1m 23s
310:	learn: 6547.4217667	total: 37.7s	remaining: 1m 23s
311:	learn: 6545.6963649	total: 37.9s	remaining: 1m 23s
312:	learn: 6541.3701212	total: 38s	remaining: 1m 23s
313:	learn: 6538.9508516	total: 38.1s	remaining: 1m 

444:	learn: 6058.3123815	total: 54.4s	remaining: 1m 7s
445:	learn: 6056.7644071	total: 54.5s	remaining: 1m 7s
446:	learn: 6053.2221949	total: 54.6s	remaining: 1m 7s
447:	learn: 6050.6867141	total: 54.8s	remaining: 1m 7s
448:	learn: 6049.2720092	total: 54.9s	remaining: 1m 7s
449:	learn: 6045.7991012	total: 55s	remaining: 1m 7s
450:	learn: 6041.8416610	total: 55.2s	remaining: 1m 7s
451:	learn: 6037.2694748	total: 55.3s	remaining: 1m 7s
452:	learn: 6035.5433403	total: 55.4s	remaining: 1m 6s
453:	learn: 6031.9625936	total: 55.5s	remaining: 1m 6s
454:	learn: 6028.6745323	total: 55.7s	remaining: 1m 6s
455:	learn: 6027.4774274	total: 55.8s	remaining: 1m 6s
456:	learn: 6024.5666364	total: 55.9s	remaining: 1m 6s
457:	learn: 6022.4674384	total: 56s	remaining: 1m 6s
458:	learn: 6020.9348900	total: 56.1s	remaining: 1m 6s
459:	learn: 6018.9248814	total: 56.3s	remaining: 1m 6s
460:	learn: 6015.1977541	total: 56.4s	remaining: 1m 5s
461:	learn: 6010.8178937	total: 56.5s	remaining: 1m 5s
462:	learn: 60

595:	learn: 5695.8419894	total: 1m 13s	remaining: 49.8s
596:	learn: 5693.1753666	total: 1m 13s	remaining: 49.7s
597:	learn: 5692.1841484	total: 1m 13s	remaining: 49.6s
598:	learn: 5690.6461813	total: 1m 13s	remaining: 49.5s
599:	learn: 5687.8173483	total: 1m 14s	remaining: 49.3s
600:	learn: 5687.1013638	total: 1m 14s	remaining: 49.2s
601:	learn: 5685.8228456	total: 1m 14s	remaining: 49.1s
602:	learn: 5682.4017894	total: 1m 14s	remaining: 49s
603:	learn: 5679.5790864	total: 1m 14s	remaining: 48.8s
604:	learn: 5678.5849027	total: 1m 14s	remaining: 48.8s
605:	learn: 5677.0006518	total: 1m 14s	remaining: 48.6s
606:	learn: 5674.8967300	total: 1m 14s	remaining: 48.5s
607:	learn: 5673.6439384	total: 1m 15s	remaining: 48.4s
608:	learn: 5670.8509876	total: 1m 15s	remaining: 48.3s
609:	learn: 5668.9358218	total: 1m 15s	remaining: 48.2s
610:	learn: 5667.3417760	total: 1m 15s	remaining: 48.1s
611:	learn: 5666.2279883	total: 1m 15s	remaining: 47.9s
612:	learn: 5665.1188904	total: 1m 15s	remaining: 

...

457:	learn: 5873.6476457	total: 55.4s	remaining: 1m 5s
458:	learn: 5871.9525700	total: 55.5s	remaining: 1m 5s
459:	learn: 5869.0891501	total: 55.6s	remaining: 1m 5s
460:	learn: 5867.7302997	total: 55.7s	remaining: 1m 5s
461:	learn: 5866.7197140	total: 55.8s	remaining: 1m 5s
462:	learn: 5863.4843169	total: 56s	remaining: 1m 4s
463:	learn: 5860.5345345	total: 56.1s	remaining: 1m 4s
464:	learn: 5856.3555717	total: 56.2s	remaining: 1m 4s
465:	learn: 5854.6049760	total: 56.4s	remaining: 1m 4s
466:	learn: 5852.3437097	total: 56.5s	remaining: 1m 4s
467:	learn: 5848.7217242	total: 56.6s	remaining: 1m 4s
468:	learn: 5847.5131032	total: 56.8s	remaining: 1m 4s
469:	learn: 5843.6748970	total: 56.9s	remaining: 1m 4s
470:	learn: 5840.2557328	total: 57s	remaining: 1m 3s
471:	learn: 5835.8701026	total: 57.1s	remaining: 1m 3s
472:	learn: 5831.5342495	total: 57.2s	remaining: 1m 3s
473:	learn: 5828.5725592	total: 57.3s	remaining: 1m 3s
474:	learn: 5826.9839891	total: 57.5s	remaining: 1m 3s
475:	learn: 58

607:	learn: 5519.5229338	total: 1m 14s	remaining: 48s
608:	learn: 5514.7839555	total: 1m 14s	remaining: 47.9s
609:	learn: 5513.4548596	total: 1m 14s	remaining: 47.8s
610:	learn: 5511.8848148	total: 1m 14s	remaining: 47.7s
611:	learn: 5507.3509856	total: 1m 15s	remaining: 47.6s
612:	learn: 5506.1962848	total: 1m 15s	remaining: 47.5s
613:	learn: 5505.4781623	total: 1m 15s	remaining: 47.3s
614:	learn: 5503.4603555	total: 1m 15s	remaining: 47.2s
615:	learn: 5502.2895115	total: 1m 15s	remaining: 47.1s
616:	learn: 5500.0425785	total: 1m 15s	remaining: 47s
617:	learn: 5497.5270104	total: 1m 15s	remaining: 46.8s
618:	learn: 5495.8152057	total: 1m 15s	remaining: 46.7s
619:	learn: 5493.9526473	total: 1m 16s	remaining: 46.6s
620:	learn: 5491.3873072	total: 1m 16s	remaining: 46.5s
621:	learn: 5488.6165409	total: 1m 16s	remaining: 46.4s
622:	learn: 5485.1465117	total: 1m 16s	remaining: 46.3s
623:	learn: 5483.5004417	total: 1m 16s	remaining: 46.1s
624:	learn: 5481.9282663	total: 1m 16s	remaining: 46

754:	learn: 5266.3624290	total: 1m 33s	remaining: 30.3s
755:	learn: 5265.2352908	total: 1m 33s	remaining: 30.2s
756:	learn: 5262.9108334	total: 1m 33s	remaining: 30.1s
757:	learn: 5261.3665397	total: 1m 33s	remaining: 30s
758:	learn: 5260.1278648	total: 1m 33s	remaining: 29.8s
759:	learn: 5258.3569086	total: 1m 34s	remaining: 29.7s
760:	learn: 5256.0142311	total: 1m 34s	remaining: 29.6s
761:	learn: 5252.9745120	total: 1m 34s	remaining: 29.5s
762:	learn: 5250.9755301	total: 1m 34s	remaining: 29.3s
763:	learn: 5249.9437698	total: 1m 34s	remaining: 29.2s
764:	learn: 5248.4979210	total: 1m 34s	remaining: 29.1s
765:	learn: 5245.5121694	total: 1m 34s	remaining: 29s
766:	learn: 5243.7183198	total: 1m 34s	remaining: 28.8s
767:	learn: 5242.9673101	total: 1m 35s	remaining: 28.7s
768:	learn: 5241.7658140	total: 1m 35s	remaining: 28.6s
769:	learn: 5239.7455228	total: 1m 35s	remaining: 28.5s
770:	learn: 5237.4146954	total: 1m 35s	remaining: 28.3s
771:	learn: 5236.9494280	total: 1m 35s	remaining: 28

902:	learn: 5056.1950892	total: 1m 51s	remaining: 12s
903:	learn: 5054.9039250	total: 1m 51s	remaining: 11.9s
904:	learn: 5054.5072606	total: 1m 51s	remaining: 11.7s
905:	learn: 5052.9424175	total: 1m 52s	remaining: 11.6s
906:	learn: 5051.2330254	total: 1m 52s	remaining: 11.5s
907:	learn: 5050.1993817	total: 1m 52s	remaining: 11.4s
908:	learn: 5049.0786681	total: 1m 52s	remaining: 11.3s
909:	learn: 5047.8292533	total: 1m 52s	remaining: 11.1s
910:	learn: 5046.4269966	total: 1m 52s	remaining: 11s
911:	learn: 5045.6026381	total: 1m 52s	remaining: 10.9s
912:	learn: 5044.7699740	total: 1m 52s	remaining: 10.8s
913:	learn: 5044.1053259	total: 1m 53s	remaining: 10.6s
914:	learn: 5043.7410137	total: 1m 53s	remaining: 10.5s
915:	learn: 5043.2264083	total: 1m 53s	remaining: 10.4s
916:	learn: 5042.4194795	total: 1m 53s	remaining: 10.3s
917:	learn: 5041.3481544	total: 1m 53s	remaining: 10.1s
918:	learn: 5040.7254434	total: 1m 53s	remaining: 10s
919:	learn: 5039.5465471	total: 1m 53s	remaining: 9.89

array([0.95211745, 0.95114004, 0.92998084, 0.95216012, 0.93137569])

In [ ]:
ctbst_gs.save_model('model_score_09237.cbm')

In [192]:
grid = {'learning_rate': [0.03, 0.1,0.5, 0.8, 0.25],
        'depth': [4, 6, 10, 3, 12],
        'l2_leaf_reg': [1, 3, 5, 7, 9]
        
       }

In [193]:
ctbst2 = catboost.CatBoostRegressor(cat_features=['Тип кузова', 'Тип топлива', 'Название', 'Регион',
                                                 'Цвет', 'Привод', 'Состояние', 'Трансмиссия', "Материал салона"],
                                   od_pval=1e-5, random_strength=1, depth=4,
                                   iterations=1000, l2_leaf_reg=3, learning_rate=0.1
                                  )

In [194]:
ctbst2.fit(X_train, y_train)

0:	learn: 21093.0575683	total: 86.3ms	remaining: 1m 26s
1:	learn: 19944.4322020	total: 189ms	remaining: 1m 34s
2:	learn: 18957.3468555	total: 266ms	remaining: 1m 28s
3:	learn: 18029.7912554	total: 367ms	remaining: 1m 31s
4:	learn: 17174.0170505	total: 453ms	remaining: 1m 30s
5:	learn: 16407.0681717	total: 542ms	remaining: 1m 29s
6:	learn: 15703.8313754	total: 625ms	remaining: 1m 28s
7:	learn: 15046.1166774	total: 711ms	remaining: 1m 28s
8:	learn: 14540.7122442	total: 797ms	remaining: 1m 27s
9:	learn: 14010.9432074	total: 889ms	remaining: 1m 28s
10:	learn: 13553.1883012	total: 1.01s	remaining: 1m 31s
11:	learn: 13211.3991478	total: 1.1s	remaining: 1m 30s
12:	learn: 12812.6324174	total: 1.19s	remaining: 1m 30s
13:	learn: 12483.3274923	total: 1.27s	remaining: 1m 29s
14:	learn: 12173.7218892	total: 1.38s	remaining: 1m 30s
15:	learn: 11819.2315419	total: 1.47s	remaining: 1m 30s
16:	learn: 11533.3168697	total: 1.56s	remaining: 1m 30s
17:	learn: 11347.7405051	total: 1.65s	remaining: 1m 29s
18

151:	learn: 6964.0041695	total: 11.9s	remaining: 1m 6s
152:	learn: 6956.9099408	total: 12s	remaining: 1m 6s
153:	learn: 6955.1775128	total: 12.1s	remaining: 1m 6s
154:	learn: 6948.0683507	total: 12.1s	remaining: 1m 6s
155:	learn: 6939.0739421	total: 12.2s	remaining: 1m 6s
156:	learn: 6935.6718812	total: 12.3s	remaining: 1m 5s
157:	learn: 6919.2712619	total: 12.4s	remaining: 1m 5s
158:	learn: 6914.5236214	total: 12.4s	remaining: 1m 5s
159:	learn: 6911.5391253	total: 12.5s	remaining: 1m 5s
160:	learn: 6906.6312563	total: 12.6s	remaining: 1m 5s
161:	learn: 6898.2263779	total: 12.6s	remaining: 1m 5s
162:	learn: 6881.2720156	total: 12.7s	remaining: 1m 5s
163:	learn: 6876.5053623	total: 12.8s	remaining: 1m 5s
164:	learn: 6871.2889884	total: 12.9s	remaining: 1m 5s
165:	learn: 6864.1162197	total: 13s	remaining: 1m 5s
166:	learn: 6852.4814509	total: 13s	remaining: 1m 4s
167:	learn: 6842.6638835	total: 13.1s	remaining: 1m 4s
168:	learn: 6837.7838682	total: 13.2s	remaining: 1m 4s
169:	learn: 6831

304:	learn: 6195.8641085	total: 23.4s	remaining: 53.4s
305:	learn: 6192.9662956	total: 23.5s	remaining: 53.3s
306:	learn: 6189.5924747	total: 23.6s	remaining: 53.2s
307:	learn: 6187.2646269	total: 23.6s	remaining: 53.1s
308:	learn: 6182.0181365	total: 23.7s	remaining: 53.1s
309:	learn: 6178.8516293	total: 23.8s	remaining: 53s
310:	learn: 6177.6359299	total: 23.9s	remaining: 52.9s
311:	learn: 6174.4647846	total: 24s	remaining: 52.8s
312:	learn: 6173.2707416	total: 24s	remaining: 52.8s
313:	learn: 6167.7272667	total: 24.1s	remaining: 52.7s
314:	learn: 6162.7884326	total: 24.2s	remaining: 52.6s
315:	learn: 6158.9689881	total: 24.3s	remaining: 52.5s
316:	learn: 6153.5100578	total: 24.3s	remaining: 52.4s
317:	learn: 6150.8675728	total: 24.4s	remaining: 52.4s
318:	learn: 6148.4848278	total: 24.5s	remaining: 52.3s
319:	learn: 6143.4817972	total: 24.6s	remaining: 52.2s
320:	learn: 6139.1791518	total: 24.6s	remaining: 52.1s
321:	learn: 6138.5627132	total: 24.7s	remaining: 52.1s
322:	learn: 6134

456:	learn: 5759.0168643	total: 35.1s	remaining: 41.7s
457:	learn: 5756.7224622	total: 35.2s	remaining: 41.6s
458:	learn: 5756.0041996	total: 35.2s	remaining: 41.5s
459:	learn: 5753.0984868	total: 35.3s	remaining: 41.5s
460:	learn: 5749.9168476	total: 35.4s	remaining: 41.4s
461:	learn: 5745.4269305	total: 35.5s	remaining: 41.3s
462:	learn: 5742.0710585	total: 35.6s	remaining: 41.2s
463:	learn: 5741.4875393	total: 35.6s	remaining: 41.2s
464:	learn: 5739.3417152	total: 35.7s	remaining: 41.1s
465:	learn: 5738.4763118	total: 35.8s	remaining: 41s
466:	learn: 5737.2591835	total: 35.9s	remaining: 40.9s
467:	learn: 5735.1031136	total: 35.9s	remaining: 40.9s
468:	learn: 5733.2671194	total: 36s	remaining: 40.8s
469:	learn: 5732.2489065	total: 36.1s	remaining: 40.7s
470:	learn: 5729.8542595	total: 36.2s	remaining: 40.6s
471:	learn: 5728.2894668	total: 36.2s	remaining: 40.5s
472:	learn: 5726.7278251	total: 36.3s	remaining: 40.5s
473:	learn: 5724.2944789	total: 36.4s	remaining: 40.4s
474:	learn: 57

608:	learn: 5458.5268069	total: 46.6s	remaining: 29.9s
609:	learn: 5456.4893217	total: 46.7s	remaining: 29.8s
610:	learn: 5455.6972156	total: 46.7s	remaining: 29.8s
611:	learn: 5452.3577156	total: 46.8s	remaining: 29.7s
612:	learn: 5451.2616697	total: 46.9s	remaining: 29.6s
613:	learn: 5446.7807404	total: 47s	remaining: 29.5s
614:	learn: 5445.0680885	total: 47s	remaining: 29.4s
615:	learn: 5444.4246222	total: 47.1s	remaining: 29.4s
616:	learn: 5443.4897072	total: 47.2s	remaining: 29.3s
617:	learn: 5441.2819574	total: 47.3s	remaining: 29.2s
618:	learn: 5439.5017523	total: 47.4s	remaining: 29.1s
619:	learn: 5437.4318275	total: 47.4s	remaining: 29.1s
620:	learn: 5434.9550580	total: 47.5s	remaining: 29s
621:	learn: 5432.1401147	total: 47.6s	remaining: 28.9s
622:	learn: 5431.7696835	total: 47.6s	remaining: 28.8s
623:	learn: 5431.2941476	total: 47.7s	remaining: 28.7s
624:	learn: 5431.1168566	total: 47.8s	remaining: 28.7s
625:	learn: 5429.4596515	total: 47.9s	remaining: 28.6s
626:	learn: 5428

760:	learn: 5210.9746894	total: 58s	remaining: 18.2s
761:	learn: 5210.2572926	total: 58.1s	remaining: 18.2s
762:	learn: 5208.7176996	total: 58.2s	remaining: 18.1s
763:	learn: 5206.3059669	total: 58.3s	remaining: 18s
764:	learn: 5206.1521841	total: 58.4s	remaining: 17.9s
765:	learn: 5205.6472912	total: 58.5s	remaining: 17.9s
766:	learn: 5205.1142889	total: 58.5s	remaining: 17.8s
767:	learn: 5204.2382567	total: 58.6s	remaining: 17.7s
768:	learn: 5203.0175446	total: 58.7s	remaining: 17.6s
769:	learn: 5200.4563085	total: 58.8s	remaining: 17.6s
770:	learn: 5200.0600359	total: 58.8s	remaining: 17.5s
771:	learn: 5198.7733927	total: 58.9s	remaining: 17.4s
772:	learn: 5197.0236984	total: 59s	remaining: 17.3s
773:	learn: 5192.1537599	total: 59.1s	remaining: 17.3s
774:	learn: 5190.2709887	total: 59.2s	remaining: 17.2s
775:	learn: 5190.0206796	total: 59.3s	remaining: 17.1s
776:	learn: 5189.2810596	total: 59.4s	remaining: 17s
777:	learn: 5187.3010097	total: 59.4s	remaining: 17s
778:	learn: 5186.909

912:	learn: 5012.3261495	total: 1m 10s	remaining: 6.67s
913:	learn: 5011.1369826	total: 1m 10s	remaining: 6.6s
914:	learn: 5009.9797559	total: 1m 10s	remaining: 6.52s
915:	learn: 5008.5768452	total: 1m 10s	remaining: 6.45s
916:	learn: 5007.8019704	total: 1m 10s	remaining: 6.37s
917:	learn: 5007.5061414	total: 1m 10s	remaining: 6.29s
918:	learn: 5006.6092478	total: 1m 10s	remaining: 6.22s
919:	learn: 5005.7312164	total: 1m 10s	remaining: 6.14s
920:	learn: 5003.8852051	total: 1m 10s	remaining: 6.06s
921:	learn: 5003.2149111	total: 1m 10s	remaining: 5.98s
922:	learn: 5003.1205181	total: 1m 10s	remaining: 5.91s
923:	learn: 5002.0889157	total: 1m 10s	remaining: 5.83s
924:	learn: 5001.5518818	total: 1m 10s	remaining: 5.75s
925:	learn: 5001.3176726	total: 1m 11s	remaining: 5.68s
926:	learn: 5000.2990499	total: 1m 11s	remaining: 5.6s
927:	learn: 4997.4150746	total: 1m 11s	remaining: 5.52s
928:	learn: 4996.7473563	total: 1m 11s	remaining: 5.45s
929:	learn: 4994.9684242	total: 1m 11s	remaining: 

In [195]:
r2_score(y_test, ctbst2.predict(X_test))

0.944392004491573